## validators


In [138]:
# Since we are adding food's to variable set too
# we should check which variable is what
def isName(var):
  return var in ['Chris', 'Brian', 'Amber', 'Diane']

def isFood(var):
  return var in ['Fries', 'Ginger', 'Hummus', 'Egg Roll']

# returs the value of variable if assigned, returns -1 otherwise
def checkValue(assignments, key):
  for assignment in assignments:
    if assignment['variable'] == key:
      return assignment['value']
  
  return -1;

# checks the specified value is assigned before
def isAssignedBefore(assignments, val, food):
  for assignment in assignments:
    if (food):
      if (isFood(assignment['variable'])):
        if assignment['value'] == val:
          return True
    else:
      if (isName(assignment['variable'])):
        if assignment['value'] == val:
          return True
  
  return False

# Appliying constraints for each variable
# Applies Amber's constraints
def amberValidator(assignments, value):
 
  if (isAssignedBefore(assignments, value, False)): 
    return False
  val = checkValue(assignments, 'Brian')
  if (val != -1):
    if (value + 5 != val):
      return False
  
  return True

# Applies Brian's constraints
def brianValidator(assignments, value):
  if (isAssignedBefore(assignments,value, False)): 
    return False

  if (value == 280):
    return False
  
  if (value == 270):
    return False

  val  = checkValue(assignments, 'Amber')
  if (val != -1):
    if (val - 5 != value):
      return False

  val1  = checkValue(assignments, 'Hummus')
  if (val1 != -1):
    if (val1 == 280):
      val2 = checkValue(assignments, 'Fries')
      if (val2 != -1):
        if (val2 != value):
          return False

  val1  = checkValue(assignments, 'Fries')
  if (val1 != -1):
    if (val1 == 280):
      val2 = checkValue(assignments, 'Hummus')
      if (val2 != -1):
        if (val2 != value):
          return False
  
  return True

# Applies Chris's constraints
def chrisValidator(assignments, value):
  if (isAssignedBefore(assignments,value, False)): return False
  if (value == 280):
    return False
  
  val = checkValue(assignments, 'Ginger')

  if (val != -1):
    if (val == value):
      return False

  return True

# Applies Diane's constraints
def dianeValidator(assignments, value):
  if (isAssignedBefore(assignments, value, False)): return False
  val  = checkValue(assignments, 'Hummus')
  if (val != -1):
    if (val + 5 != value):
      return False
  return True

# Applies Hummus's constraints
def hummusValidator(assignments, value):
  if (isAssignedBefore(assignments, value, True)): return False
 
  val  = checkValue(assignments, 'Diane')
  if (val != -1):
    if (val - 5 != value):
      return False

  val1  = checkValue(assignments, 'Fries')
  if (val1 != -1):
    if (val1 == value):
      return False
    else:
      if (val1 != 280):
        if (value != 280):
          return False

  val1  = checkValue(assignments, 'Brian')
  if (val1 != -1):
    if (value not in [280, val1]):
      return False
  
  return True

# Applies Fries's constraints
def friesValidator(assignments, value):
  
  if (isAssignedBefore(assignments, value, True)): return False
  val1  = checkValue(assignments, 'Hummus')
  if (val1 != -1):
    if (val1 == value):
      return False
    else:
      if (val1 != 280):
        if (value != 280):
          return False

  val1  = checkValue(assignments, 'Brian')
  if (val1 != -1):
    if (value not in [280, val1]):
      return False

  
  return True

# Applies Ginger Bread's constraints
def gingerBreadValidator(assignments, value):
  if (isAssignedBefore(assignments,value, True)): return False
  if (value == 280):
    return False

  val  = checkValue(assignments, 'Chris')
  if (val != -1):
    if (val == value):
      return False

  return True

# Applies Egg Roll's constraints
def eggrollValidator(assignments, value):
  if (isAssignedBefore(assignments, value, True)): return False
  return True
  

## Functions

In [100]:
# Finds next variable to assign first
def find_var_mrv(domains, assignments):

  currentMinLen = 999
  minVars = []

  for var, domain in domains.items():
    # if already assigned continue
    if (checkValue(assignments, var) != -1): continue

    # find the variable with minimum domain length

    if (len(domain) < currentMinLen):
      minVars.clear()
      currentMinLen = len(domain)
      minVars.append(var)
    
    elif len(domain) == currentMinLen:
      minVars.append(var)

  # sort them according to their names for in case of tie
  minVars.sort()
  return minVars[0]


In [144]:
# Finds the value to assign to the selected variable
def find_val_lcv(constraints, domains,assignments, variable):
  # Create copies of domains and assignments for test assignment
  dTemp = {}

  aTemp = [x for x in assignments]

  for k, v in domains.items():
    dTemp[k] = [x for x in v]

  # values and their lcv rates, bigger the better
  falseData = {
      
  }

  # iterate over domain of current variable
  for v in dTemp[variable]:

    # test assignment
    assignment = {
        'variable': variable,
        'value': v
    }

    aTemp.append(assignment)

    for var, domain in dTemp.items():
      if (var == variable): continue
      #if (isFood(var)): continue
      
      # check every domain and find their remaining domains after test assignment
      for value in domain:
        isValid = constraints[var](aTemp, value)

        if (not isValid):
          domain.remove(value)
          if (len(domain) == 0):
            try: 
              del falseData[v]
              break
            except:break

        currentFalseData = falseData.get(v, 0)
        falseData[v] = currentFalseData + len(domain)
        

    aTemp.remove(assignment)

  currentLeastValue = 0
  currentLeastVar = 0


  # find the lcv in the 'falseData'
  for k,v in falseData.items():
    if (v > currentLeastValue):
      currentLeastVar = k
      currentLeastValue = v


  if (currentLeastVar == 0):
    return dTemp[variable][0]
  
  return currentLeastVar

In [102]:
# applies arc consistency
def apply_arc_consistency(constraints,domains, assignments):

  newDomains = {}

  for var, domain in domains.items():
    newDomain = []
  
    for value in domain:
      # if assignment is not valid, remove it from the domain (not add to the new)
      isValid = constraints[var](assignments, value)
      if (isValid):
        newDomain.append(value)
    
    newDomains[var] = newDomain
  
  return newDomains


In [146]:
# checks the whether current state is solution or not 
def isSolution(domains, assignments, variables):
  # all variables are assigned, solution found.
  if (len(assignments) == len(variables)):
    return True
  
  n = 0

  # every domain is either empty or of length 1, solution found.
  for var, domain in domains.items():
    if (len(domain) <= 1):
      n += 1
  
  if ( len(domains.keys()) == n):
    return True
  
  # Still no solution found
  return False

def hasNoSolution(domains, assignments, variables):

  nonEmptyDomainCount = 0

  for var, domain in domains.items():
    if (len(domain) >= 1):
      nonEmptyDomainCount += 1
  
  # all domains are empty and still there is not enough assignments, so no solution
  if (nonEmptyDomainCount == 0 and (len(assignments) != len(variables))):
    return True
  
  return False


In [105]:
# if solution found before all assignments are done, assigns remainin values.
def assignRemaining(domains, assignments):

  newAssignments = [x for x in assignments]

  for var, domain in domains.items():
    if (len(domain) == 1):  
      newAssignment = {
          'variable': var,
          'value': domain[0]
      }

      newAssignments.append(newAssignment)

  return newAssignments


In [121]:
# prints current state of assignments and domains
def displayInfo(domains, assignments):
  print("")
  print("")
  print("== == == == == == == == == == == == == == = == =")
  print("== == == == == == == == == == == == == == = == =")
  print("Domains")
  print('----------------------------------')
  for var, domain in domains.items():
  
    print(f"{var}: {' - '.join([str(x) for x in domain])}")
  print("== == == == == == == == == == == == == == = == =")
  print("")
  print("Assignments")
  print('----------------------------------')
  if (len(assignments) == 0):
    print("No assignment yet")
  for a in assignments:
    print(f"{a['variable']}: {a['value']}")

  print("== == == == == == == == == == == == == == = == =")
  print("== == == == == == == == == == == == == == = == =") 
  print("")
  print("")

  pass

In [143]:
timeMap = {
    270: '4.30',
    275: '4.35',
    280: '4.40',
    285: '4.45'
}
# displays the last solution in readeble way.
def displaySolution(assignments):
  finalData = {}
  for assignment in assignments:
    var = assignment['variable']
    val = assignment['value']

    current = finalData.get(val, {})

    if (isName(var)):
      current['person'] = var
    else:
      current['food'] = var
      
    finalData[val] = current

  for time, data in finalData.items():
    print(f"{data['person']} brought {data['food']} at {timeMap[time]}")



## Main

In [148]:
X = ["Brian", "Amber", "Chris", "Diane", "Hummus", "Egg Roll", "Fries", "Ginger"]

assignments = []
domains = {}

# Default domain, all variables has the same domain
#.    4.30 - 4.35 - 4.40 - 4.45
D =  [270,   275,    280,  285]

for v in X:
  domains[v] = [x for x in D]

# Constrains map, every variable is mapped to validators above.
C = {
  "Brian":brianValidator,
  "Amber":amberValidator,
  "Chris":chrisValidator,
  "Diane":dianeValidator,
  "Hummus":hummusValidator,
  "Egg Roll":eggrollValidator,
  "Fries":friesValidator,
  "Ginger":gingerBreadValidator
}

# Displays the initial state.
displayInfo(domains, assignments)

# initial arc consistency check
domains = apply_arc_consistency(C, domains, assignments)

# if solution reached even after first arc check
if (isSolution(domains, assignments, X)):
  assignRemaining(domains, assignments)
  displaySolution(assignments)

# First variable to assign
firstToAssign = find_var_mrv(domains, assignments)
# First value to assign to the first variable
valueToAssign = find_val_lcv(C, domains, assignments, firstToAssign)

firstAssignment= {
    'variable': firstToAssign,
    'value': valueToAssign
}

# initial assignment
assignments.append(firstAssignment)

# arc consistency after initial assignment
domains = apply_arc_consistency(C, domains, assignments)

# if solution reached even after second arc check
if (isSolution(domains, assignments, X)):
  assignRemaining(domains, assignments)
  displaySolution(assignments)

noSolution = False

# Main loop
while True:
  
  var = find_var_mrv(domains,assignments)
  val = find_val_lcv(C, domains, assignments, var)

  del domains[var]

  a = {
    'variable': var,
    'value': val
  }

  assignments.append(a)
  domains = apply_arc_consistency(C, domains,assignments)

  if (isSolution(domains, assignments, X)):
    assignments = assignRemaining(domains, assignments)
    break
  
  if (hasNoSolution(domains, assignments, X)):
    print("No solution!")
    break

  displayInfo(domains, assignments)


print("Final State")
displayInfo(domains, assignments)

if (not noSolution):
  print("Final Result")
  displaySolution(assignments)




== == == == == == == == == == == == == == = == =
== == == == == == == == == == == == == == = == =
Domains
----------------------------------
Brian: 270 - 275 - 280 - 285
Amber: 270 - 275 - 280 - 285
Chris: 270 - 275 - 280 - 285
Diane: 270 - 275 - 280 - 285
Hummus: 270 - 275 - 280 - 285
Egg Roll: 270 - 275 - 280 - 285
Fries: 270 - 275 - 280 - 285
Ginger: 270 - 275 - 280 - 285
== == == == == == == == == == == == == == = == =

Assignments
----------------------------------
No assignment yet
== == == == == == == == == == == == == == = == =
== == == == == == == == == == == == == == = == =




== == == == == == == == == == == == == == = == =
== == == == == == == == == == == == == == = == =
Domains
----------------------------------
Brian: 
Chris: 285
Diane: 280 - 285
Hummus: 275 - 280
Egg Roll: 270 - 275 - 280 - 285
Fries: 275 - 280
Ginger: 270 - 275 - 285
== == == == == == == == == == == == == == = == =

Assignments
----------------------------------
Brian: 275
Amber: 270
== == == == == =